In [1]:
import pandas as pd
import json

## Carregar os dados

In [2]:
# Carregar o dataset JSON
with open('HPI_master.json', 'r') as f:
    json_data = json.load(f)

# Converter o dataset JSON para um DataFrame do pandas
data_json = pd.DataFrame(json_data)

# Carregar o dataset CSV - Population size
data_csv_ps = pd.read_csv("cu.data.19.PopulationSize.csv")

# Carregar o dataset CSV - 
data_csv_fb = pd.read_csv("cu.data.11.USFoodBeverage.csv")

## Visualizar os dados

In [3]:
data_json.head()

,hpi_type,hpi_flavor,frequency,level,place_name,place_id,yr,period,index_nsa,index_sa
0,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,1,100.00,100.0
1,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,2,100.91,100.96
2,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,3,101.30,100.91
3,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,4,101.69,100.98
4,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,5,102.32,101.36


In [4]:
data_csv_ps.head()

,series_id,year,period,value,footnote_codes
0,CUURA000AA0,1986,M12,100.0,NaN
1,CUURA000AA0,1987,M01,100.6,NaN
2,CUURA000AA0,1987,M02,101.1,NaN
3,CUURA000AA0,1987,M03,101.6,NaN
4,CUURA000AA0,1987,M04,102.2,NaN


In [5]:
data_csv_fb.head()

,series_id,year,period,value,footnote_codes
0,CUSR0000SAF,1967,M01,34.8,NaN
1,CUSR0000SAF,1967,M02,34.7,NaN
2,CUSR0000SAF,1967,M03,34.7,NaN
3,CUSR0000SAF,1967,M04,34.6,NaN
4,CUSR0000SAF,1967,M05,34.6,NaN


In [6]:
data_csv_ps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84760 entries, 0 to 84759
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   series_id       84760 non-null  object 
 1   year            84760 non-null  int64  
 2   period          84760 non-null  object 
 3   value           84760 non-null  float64
 4   footnote_codes  0 non-null      float64
dtypes: float64(2), int64(1), object(2)
memory usage: 3.2+ MB


Conseguimos observar a existência de dados do tipo `object`. Poderá ser necessário tratar este tipo de dados.

In [7]:
data_csv_ps.isnull().sum()

series_id             0
year                  0
period                0
value                 0
footnote_codes    84760
dtype: int64

In [8]:
data_csv_fb.isnull().sum()

series_id              0
year                   0
period                 0
value                  0
footnote_codes    119198
dtype: int64

## Tratamento dos Dados

#### **Transformar `period` em dados numéricos**

In [9]:
# Dicionário para mapear o período número ao valor da string
period_map = {
    'M01': 1,
    'M02': 2,
    'M03': 3,
    'M04': 4,
    'M05': 5,
    'M06': 6,
    'M07': 7,
    'M08': 8,
    'M09': 9,
    'M10': 10,
    'M11': 11,
    'M12': 12,
    'M13:': 13
}

data_csv_ps['period'] = data_csv_ps["period"].replace(period_map)
data_csv_fb['period'] = data_csv_fb["period"].replace(period_map)

In [10]:
data_csv_ps.head()

,series_id,year,period,value,footnote_codes
0,CUURA000AA0,1986,12,100.0,NaN
1,CUURA000AA0,1987,1,100.6,NaN
2,CUURA000AA0,1987,2,101.1,NaN
3,CUURA000AA0,1987,3,101.6,NaN
4,CUURA000AA0,1987,4,102.2,NaN


#### **Remover a coluna `footnote_codes`**

In [11]:
data_csv_ps.drop(columns=["footnote_codes"], inplace=True)
data_csv_fb.drop(columns=["footnote_codes"], inplace=True)

In [12]:
data_csv_ps.head()

,series_id,year,period,value
0,CUURA000AA0,1986,12,100.0
1,CUURA000AA0,1987,1,100.6
2,CUURA000AA0,1987,2,101.1
3,CUURA000AA0,1987,3,101.6
4,CUURA000AA0,1987,4,102.2


In [13]:
data_csv_fb.head()

,series_id,year,period,value
0,CUSR0000SAF,1967,1,34.8
1,CUSR0000SAF,1967,2,34.7
2,CUSR0000SAF,1967,3,34.7
3,CUSR0000SAF,1967,4,34.6
4,CUSR0000SAF,1967,5,34.6


## Realizar o Merge dos datasets

In [1]:
# Fazer merge dos CSV's
merged_df = pd.merge(data_csv_ps, data_csv_fb, on=['year', 'period'])

# Fazer merge do CSV com JSON
data_json.rename(columns = {'yr':'year'}, inplace = True)

# Merge the two dataframes on the common columns 'year' and 'period'
merged_data = pd.merge(merged_df, data_json, on=['year', 'period'])

# Escrever novo dataset para um ficheiro CSV
merged_data.to_csv('merged_data.csv', index=False)

# VER artigo: Pandas scaling to large datasets

NameError: name 'pd' is not defined

## Parquet

In [ ]:
import pyarrow as pa
import pyarrow.parquet as pq

#pq.write_table(table, 'population_size.parquet')
#pq.write_table(table, 'food_beverages.parquet')
#pq.write_table(table, 'hpi.parquet')